In [1]:
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
# from modelscope import snapshot_download
# model_dir = snapshot_download("ZhipuAI/chatglm3-6b", revision = "v1.0.0")
# print(model_dir)
model_dir = "/mnt/workspace/.cache/modelscope/ZhipuAI/chatglm3-6b"

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-21 13:23:29.810349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 13:23:30.807550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
# model.half()与 load_in_8bit只能二选一;load_in_8bit=True时,已经将模型送入GPU.
model = AutoModel.from_pretrained(model_dir, load_in_8bit=True, trust_remote_code=True, device_map ='auto')#.half().cuda()
model = model.eval()
history = []

Loading checkpoint shards: 100%|██████████| 7/7 [01:09<00:00,  9.92s/it]


In [3]:
!nvidia-smi

Thu Dec 21 13:25:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   33C    P0    65W / 300W |   7922MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [85]:
def inference_fn(q, max_length, LoRA_flag=False):
    """
    用于实现base model的推理
    q: query
    max_length:输出的长度
    LoRA_flag: 是否是LoRA微调之后的模型. 当为True时,需要model.base_model.generate()
    """
    input_ids = tokenizer(q, return_tensors='pt').to('cuda')
    # print(f'base_model_input_ids:\n{input_ids}')
    model.eval()
    if LoRA_flag:
        output_ids = model.base_model.generate(input_ids['input_ids'],max_length=max_length)
    else:
        output_ids = model.generate(input_ids['input_ids'],max_length=max_length)
    # print(f'base_model_output_ids:\n{output_ids}')
    response_ids = output_ids[0,input_ids['input_ids'].shape[-1]:]
    response = tokenizer.decode(response_ids,skip_special_tokens=False)
    return response

In [47]:
q = "python-查看字典的长度,如何?"

# base_model模式: 不需要role
response = inference_fn(q,max_length=8192)
print(f"base mode response:\n{response}")
# chat_model模式: 可以多role
response, history = model.chat(tokenizer, q, history=history)
response, history = model.chat(tokenizer, query=q, history= history, role= "user",  max_length = 8192, num_beams=1, do_sample=True, top_p=0.8, temperature=0.8,)
print(f"chat model response:\n{response}")

base mode response:

 可以使用 Python 内置的 `len()` 函数来查看字典的长度。下面是一个简单的示例代码:

```python
my_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
dict_length = len(my_dict)
print('字典的长度为:', dict_length)
```

在这个示例中,我们首先创建了一个名为 `my_dict` 的字典,并使用 `len()` 函数计算其长度,将其存储在变量 `dict_length` 中。最后,我们使用 `print()` 函数输出字典的长度。

输出结果应该是:

```
字典的长度为: 3
```

因为 `my_dict` 包含三个键值对。
chat model response:
在 Python 中，可以使用 `len()` 函数来查看字典的长度。下面是一个简单的例子：

```python
# 创建一个字典
my_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}

# 获取字典的长度
dict_length = len(my_dict)

# 输出字典的长度
print("字典的长度为：", dict_length)
```

在这个例子中，我们首先创建了一个名为 `my_dict` 的字典，然后使用 `len()` 函数来获取其长度，并将结果存储在变量 `dict_length` 中。最后，我们使用 `print()` 函数将字典的长度输出到控制台。


In [6]:
ids = tokenizer(q)
ids_encode = tokenizer.encode(q, add_special_tokens=True)
ids_decode = tokenizer.decode(ids_encode, skip_special_tokens=False)
vocab = tokenizer.get_vocab()
vocab_size = tokenizer.vocab_size

print(tokenizer.get_command("<pad>"))
print(tokenizer.unk_token)
print(tokenizer.pad_token)
print(f"eos的token ids: {tokenizer.eos_token_id}")
print(f"ids: {ids}")
print(f"ids_encode: {ids_encode}")
print(f"text_decode: {ids_decode}")
print(f"vocal_size: {vocab_size}")
token_id = tokenizer._convert_token_to_id('四书') 
print(f"token: {token_id}")
tokens_string= tokenizer.convert_tokens_to_string(['美丽','的','花朵'])
print(f"tokens_to_string: {tokens_string}")
print(tokenizer.get_prefix_tokens)
single_message = tokenizer.build_single_message( role='assistant', metadata='test', message=q)
print(f'build_single_message:{single_message}')
print(tokenizer.decode(single_message))
chat_input = tokenizer.build_chat_input(query=q, history=None, role="user")
print(chat_input)
print(tokenizer.decode(chat_input['input_ids'][0]))

0
<unk>
<unk>
eos的token ids: 2
ids: {'input_ids': [64790, 64792, 19079, 30941, 33367, 52679, 35465, 54648, 30932, 31750, 30987], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
ids_encode: [64790, 64792, 19079, 30941, 33367, 52679, 35465, 54648, 30932, 31750, 30987]
text_decode: [gMASK]sop python-查看字典的长度,如何?
vocal_size: 64798
token: 0
tokens_to_string: 美丽的花朵
<bound method ChatGLMTokenizer.get_prefix_tokens of ChatGLMTokenizer(name_or_path='/mnt/workspace/.cache/modelscope/ZhipuAI/chatglm3-6b', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	
}>
build_single_message:[64796, 1429, 13, 19079, 30941, 33367, 52679, 35465, 54648, 30932, 31750, 30987]
<|assistant|> test
 python-查看字典的长度,如何?
{'input_ids': tensor([[647

In [49]:
datas = [
{
    "context": "大师，我最近事业不顺利，感觉很迷茫，该怎么办？",
    "target": "孩子，生活中的起伏和挫折是常态。放下烦恼，用平静的心态面对困境，努力奋斗，同时保持内心的善良与慈悲，你将找到属于你的光明之路。",
},
{
    "context": "师父，我与家人之间常常发生争吵，如何改善家庭关系？",
    "target": "阿弥陀佛，亲情是一份宝贵的财富，要从宽容和理解开始。培养平和的心态，倾听彼此的意见，以慈悲与智慧去化解冲突，共同创造和谐温暖的家庭氛围。",
},
{
    "context": "师傅，我经常感到焦虑和压力，怎样才能获得内心的宁静？",
    "target": "孩子，内心的宁静源于觉察和自我调适。修行冥想，深呼吸，寻找内心的栖息地。明白外在的变幻无常，放下过多的执着与欲望，你将发现内心的平静与安宁。",
},
{
    "context": "大师，我常常陷入烦恼和抱怨，怎样才能培养感恩之心？",
    "target": "施主，感恩是一种修养，需要从日常小事开始。每天意识到自己的幸运和获得，体察他人的善意和关爱。珍惜当下，感恩生活中的点滴，你将发现更多的快乐与满足。",
},
{
    "context": "师父，我对未来感到迷茫和恐惧，不知道自己的人生目标是什么，有何建议？",
    "target": "阿弥陀佛，人生的方向不一定要一目了然，它是通过不断尝试和探索而逐渐清晰的。保持对未知的勇敢与开放，寻找自己内心的声音。用心去体验，用智慧去选择，你将找到属于自己的道路。",
}]

max_prompt_len = max([len(data['context']) for data in datas]) + 5
max_response_len = max([len(data['target']) for data in datas]) + 5
max_seq_length = max_prompt_len + max_response_len

[<font color=lime size=6 face=lisu>github: ChatGLM3-6B 微调示例</font>](https://github.com/THUDM/ChatGLM3/tree/main/finetune_chatmodel_demo)

In [53]:
from typing import Dict, List
from transformers import PreTrainedTokenizer, DataCollatorForSeq2Seq
from torch.utils.data import Dataset

class InputOutputDataset(Dataset):
    def __init__(self, data: List[dict], tokenizer: PreTrainedTokenizer, max_source_length: int, max_target_length: int):
        super(InputOutputDataset, self).__init__()
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        self.max_seq_length = max_source_length + max_target_length + 1
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i) -> dict:
        data_item = self.data[i]

        a_ids = self.tokenizer.encode(text=data_item['context'], add_special_tokens=True, truncation=True,
                                         max_length=self.max_source_length)
        b_ids = self.tokenizer.encode(text=data_item['target'], add_special_tokens=False, truncation=True,
                                    max_length=self.max_target_length)

        context_length = len(a_ids)
        input_ids = a_ids + b_ids + [self.tokenizer.eos_token_id]
        labels = [self.tokenizer.pad_token_id] * context_length + b_ids + [self.tokenizer.eos_token_id]
        
        pad_len = self.max_seq_length - len(input_ids)
        input_ids = input_ids + [self.tokenizer.pad_token_id] * pad_len
        labels = labels + [self.tokenizer.pad_token_id] * pad_len
        labels = [(l if l != self.tokenizer.pad_token_id else -100) for l in labels]

        assert len(input_ids) == len(labels), f"length mismatch: {len(input_ids)} vs {len(labels)}"

        return {
            "input_ids": input_ids,
            "labels": labels
        }

In [54]:
dataset = InputOutputDataset(data=datas, tokenizer=tokenizer, max_source_length=max_prompt_len, max_target_length=max_response_len)
# dataset = MultiTurnDataset(data=datas, tokenizer=tokenizer, max_seq_length=max_seq_length)
print(dataset[0])
print(tokenizer.decode(dataset[0]['input_ids']))

{'input_ids': [64790, 64792, 33697, 54758, 31123, 54546, 32483, 32018, 54535, 32790, 31123, 32044, 54657, 40739, 31123, 49086, 31514, 30910, 31652, 31123, 37818, 38907, 54542, 38625, 54532, 34611, 31155, 35685, 35644, 31123, 54571, 37129, 38888, 32122, 36198, 31123, 31862, 32721, 31123, 31701, 31983, 37005, 35178, 54619, 42049, 31123, 54622, 54687, 32523, 32180, 31822, 34702, 33532, 31155, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 31652, 31123, 37818, 38907, 54542, 38625, 54532, 34611, 31155, 35685, 35644, 31123, 54571, 37129, 38888, 32122, 36198, 31123, 31862, 32721, 31123, 31701, 31983, 37005, 35178, 54619, 42049, 31123, 54622, 54687, 32523, 32180, 31822, 34702, 33532, 31155, 2, -100, 

In [57]:
from peft import LoraConfig, TaskType, get_peft_model,prepare_model_for_kbit_training

# 确保梯度检查点和模型并行化设置正确
#model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.is_parallelizable = True
model.model_parallel = True  # 可以尝试暂时关闭模型并行化来看是否解决问题
# model.lm_head = CastOutputToFloat(model.transformer.output_layer)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!


model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['query_key_value'],
    inference_mode=False,
       )

model = get_peft_model(model, peft_config).to('cuda')

training_args = TrainingArguments(
    "output",
    # fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 2,
    learning_rate = 1e-4,
    num_train_epochs=100,
    logging_steps=10,
    remove_unused_columns=False,
    label_names=['labels'],
    seed=0,
    data_seed=0,
    group_by_length=False,
)

# Data collator
data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=-100,
        pad_to_multiple_of=None,
        padding=False
)

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=data_collator,
)


In [58]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,3.708600
20,2.759700
30,1.827000
40,1.035100
50,0.357100
60,0.064400
70,0.012000
80,0.004800
90,0.002700
100,0.002000


TrainOutput(global_step=450, training_loss=0.2176804869999695, metrics={'train_runtime': 484.0246, 'train_samples_per_second': 1.55, 'train_steps_per_second': 0.93, 'total_flos': 3470922925056000.0, 'train_loss': 0.2176804869999695, 'epoch': 150.0})

In [92]:
model.config.use_cache = True
history=[]
q = "大师，我会为与同事的关系相处而苦恼,如何改善与同事的关系呢?"

# base_model模式: 不需要role
response = inference_fn(q,max_length=1024, LoRA_flag=True)
print(f"base mode response:\n{response}")
# chat_model模式: 可以多role
response, history = model.chat(tokenizer, q, history=history)
response, history = model.chat(tokenizer, query=q, history= history, role= "user",  max_length = 8192, num_beams=1, do_sample=True, top_p=0.8, temperature=0.8,)
print(f"chat model response:\n{response}")

base mode response:
孩子，我会告诉你一些与同事相处的技巧。首先，要诚实、友好和尊重他人。其次，要善于倾听他人的意见，并学会表达自己的观点。最后，要勇于承担责任，并尝试与他人建立良好的团队关系。
chat model response:
以下是一些建议:

1. 积极沟通:沟通是建立良好关系的关键。尝试与同事进行积极的对话,分享自己的想法和观点,并听取他们的看法。如果存在分歧,尝试以合作的方式解决问题。

2. 尊重他人:尊重是建立良好关系的基础。尊重同事的意见和观点,不要对他们的想法进行批评或攻击。同时,也要尊重同事的隐私和个人空间。

3. 诚实坦率:诚实是建立信任和良好关系的关键。尝试与同事坦诚相待,分享自己的想法和感受,同时也要接受他们的反馈和建议。

4. 建立共同目标:与同事建立共同的目标和价值观可以加强团队凝聚力和合作精神。尝试与同事一起制定目标并努力实现它们。

5. 学会妥协:在人际关系中,妥协是必要的。尝试与同事一起找到一个共同的解决方案,以满足双方的需求和利益。

6. 培养团队合作精神:团队合作精神是建立良好关系的关键。尝试与同事一起工作,分享资源和知识,以实现更好的结果。

7. 尊重不同的文化和背景:如果同事来自不同的文化或背景,尝试理解他们的文化和价值观。尊重他们的差异,并尝试与他们在相互尊重的基础上建立关系。

改善与同事的关系需要积极的努力和沟通。尝试遵循这些建议,以建立更好的工作关系。
